# IST736 Proj Gloomy Authors SVM and MNB Comparison

## Using whole data and seperate them 20%test and 80% train

In [70]:
import pandas as pd
train=pd.read_csv(r"C:\Users\wangtao\Desktop\data.csv",encoding='latin1')
y=train['author'].values
X=train['text'].values

In [77]:
# the whole data seperated into two parts 
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

(16575,) (16575,) (4144,) (4144,)
That's not love and you know it Mel said."
RC
Terri hon don't look that way."
RC


In [82]:
# Checking the data and I find some 'NAN's then I finde them and deleted 
#import numpy as np
training_labels = set(y_train)
print(training_labels)
from scipy.stats import itemfreq
test_category_dist = itemfreq(y_test)
print(test_category_dist)
#training_catagory_dist = np.unique(y_train)
#print(training_category_dist)

{'RY', 'JJ', 'RC'}
[['JJ' 1502]
 ['RC' 1174]
 ['RY' 1468]]


C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  


In [50]:
import numpy as np
df=pd.DataFrame(train)
print(np.any(df.isnull())==True)#打印出至少一个丢失了的数据  另外 简单的表格 可以用 df.isnull() pirnt 出来检查
#df.fillna(value=0) 替换掉nan  
print((np.where(df.isnull()==True))[0])
df.iloc[(np.where(df.isnull()==True))[0]]#打印Nan 所在

False
[]


,chapter,author,text


In [83]:
from sklearn.feature_extraction.text import CountVectorizer
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5,stop_words='english')
bigram_count_vectoritzer = TfidfVectorizer(encoding='latin-1',binary=False, ngram_range=(1,2),min_df=5,stop_words='english')

In [84]:

# fit vocabulary in training documents and transform the training documents into vectors
#X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
X_train_vec = bigram_count_vectorizer.fit_transform(X_train)

# check the content of a document vector
print(X_train_vec.shape)
print(X_train_vec[0].toarray())

# check the size of the constructed vocabulary
#print(len(unigram_count_vectorizer.vocabulary_))

# print out the first 10 items in the vocabulary
#print(list(unigram_count_vectorizer.vocabulary_.items())[:10])
print(list(bigram_count_vectorizer.vocabulary_.items())[:50])

(16575, 3488)
[[0 0 0 ... 0 0 0]]
3488
[('kinda', 1666), ('hotel', 1483), ('partner', 2193), ('open', 2119), ('expression', 1020), ('interested', 1566), ('knock', 1681), ('compartment', 614), ('free', 1189), ('sullen', 3013)]


In [85]:
#Vectorize the test data
#X_test_vec = unigram_count_vectorizer.transform(X_test)
X_text_vect = bigram_count_vectorizer.tranform(X_test)
# print out #examples and #features in the test set
print(X_test_vec.shape)

(4144, 3488)


## Train MNB Classifier

In [86]:
#Train MNB
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB()
nb_clf.fit(X_train_vec, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [89]:
feature_ranks = sorted(zip(nb_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))
JJ_10 = feature_ranks[-10:]
for i in range(0, len(JJ_10)):
    print(JJ_10[i])
print()

feature_ranks = sorted(zip(nb_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))
not_JJ_10 = feature_ranks[:10]
for i in range(0, len(not_JJ_10)):
    print(not_JJ_10[i])
print()

(-5.305838516606873, 'thing')
(-5.202460162153039, 'don')
(-5.198005811803658, 'just')
(-5.184760585053637, 'right')
(-5.176026905084883, 'way')
(-5.1545206998639195, 'know')
(-5.116937988714876, 'little')
(-4.930526446669397, 'like')
(-4.903679196633209, 'time')
(-3.942073268547011, 'said')

(-10.614106214008078, 'abroad')
(-10.614106214008078, 'accident')
(-10.614106214008078, 'accompanist')
(-10.614106214008078, 'account')
(-10.614106214008078, 'acts')
(-10.614106214008078, 'adventure')
(-10.614106214008078, 'aid')
(-10.614106214008078, 'alarmed')
(-10.614106214008078, 'albert')
(-10.614106214008078, 'alittle')



In [90]:
nb_clf.score(X_test_vec,y_test)

0.7719594594594594

In [98]:
from sklearn.metrics import confusion_matrix
y_pred = nb_clf.predict(X_test_vec)
cm = confusion_matrix(y_test,y_pred,labels=['JJ','RY','RC'])
print(cm)

from sklearn.metrics import classification_report
target_names = ['JJ','RY','RC']
print(classification_report(y_test,y_pred,target_names=target_names))

[[1229  155  118]
 [ 177 1132  159]
 [ 156  180  838]]
             precision    recall  f1-score   support

         JJ       0.79      0.82      0.80      1502
         RY       0.75      0.71      0.73      1174
         RC       0.77      0.77      0.77      1468

avg / total       0.77      0.77      0.77      4144



## Train LinearSVC Classifier 

In [100]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1)
svm_clf.fit(X_train_vec,y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [105]:
#feature_rank = sorted(zip(svm_clf.coef_[0],unigram_count_vectorizer.get_feature_names()))
feature_rank = sorted (zip(svm_clf.coef_[0],bigram_count_vectorizer.get_feature_names()))
a=feature_rank[-50:]
for i in range(0,len(a)):
    print(a[i])
print()

(1.826690215305478, 'lenehan')
(1.8521491832420724, 'blouse')
(1.8761165307688232, 'chandler')
(1.8838378129205338, 'colour')
(1.9533384270038006, 'chief')
(1.9571143528336608, 'maria')
(1.9738715267332492, 'sing')
(2.0836284261210185, 'briton')
(2.12512981472834, 'gabriel')
(2.4683480581091266, 'corley')



In [107]:
svm_clf.score(X_test_vec,y_test)

0.8211872586872587

In [109]:
from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm = confusion_matrix(y_test,y_pred,labels=['JJ','RY','RC'])
print(cm)

from sklearn.metrics import classification_report
target_names = ['JJ','RY','RC']
print(classification_report(y_test,y_pred,target_names=target_names))

[[1257   77  168]
 [ 100 1157  211]
 [  77  108  989]]
             precision    recall  f1-score   support

         JJ       0.88      0.84      0.86      1502
         RY       0.72      0.84      0.78      1174
         RC       0.86      0.79      0.82      1468

avg / total       0.83      0.82      0.82      4144

